SKRIPSI

# Import library

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk

from nltk import ngrams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, validation_curve,RandomizedSearchCV

np.random.seed(0)
stopwords = nltk.corpus.stopwords.words('english')
poster_Stemmer = nltk.PorterStemmer()


# Baca file csv

In [2]:
#baca csv
data_train_amazon = pd.read_csv('Amazon_Train.csv')
data_train_yelp = pd.read_csv('Yelp_Train.csv')
data_test_amazon = pd.read_csv('Amazon_Test.csv')
data_test_yelp = pd.read_csv('Yelp_Test.csv')

In [3]:
print(data_train_amazon.head())
print("\n")
print(data_train_yelp.head())

   Label                                           Sentimen
0      0  Buyer beware This is a self-published book, an...
1      0  The Worst! A complete waste of time. Typograph...
2      0  Oh please I guess you have to be a romance nov...
3      0  Awful beyond belief! I feel I have to write to...
4      0  Another Abysmal Digital Copy Rather than scrat...


   Label                                           Sentimen
0      0  I don't know what Dr. Goldberg was like before...
1      0  I'm writing this review to give you a heads up...
2      0  Owning a driving range inside the city limits ...
3      0  This place is absolute garbage...  Half of the...
4      0  Used to go there for tires, brakes, etc.  Thei...


In [4]:
print(data_test_amazon.head())
print("\n")
print(data_test_yelp.head())

   Label                                           Sentimen
0      0  Overly complicated Being both a U.S. history b...
1      0  Terrible Disappointment -- parts don't fit My ...
2      0  Didn't hold up.....very disappointed I bought ...
3      0  gene hates jezebel i love JLJ but this compila...
4      0  Nice toy but ... My six-year-old loves space a...


   Label                                           Sentimen
0      0  My wife and I used to love Arriba's, til recen...
1      0  You get what you pay for.  The food is inexpen...
2      0  Unfortunately, yesterday's visit was one of th...
3      0  I went into the Scottsdale location yesterday....
4      0  It takes a lot for me to write a review and bl...


# Preprocessing ke lowercase

In [5]:
iterator = 0
data_Preprocessing_Amazon_Train = data_train_amazon
data_Lowercase_Amazon_Train = []

data_Preprocessing_Yelp_Train = data_train_yelp
data_Lowercase_Yelp_Train = []

while iterator < len(data_train_amazon) :
    data_Lowercase_Amazon_Train.append(data_train_amazon.Sentimen[iterator].lower())
    iterator = iterator + 1

iterator = 0

while iterator < len(data_train_yelp) :
    data_Lowercase_Yelp_Train.append(data_train_yelp.Sentimen[iterator].lower())
    iterator = iterator + 1
    

In [6]:
iterator = 0
data_Preprocessing_Amazon_Test = data_test_amazon
data_Lowercase_Amazon_Test = []

data_Preprocessing_Yelp_Test = data_test_yelp
data_Lowercase_Yelp_Test = []

while iterator < len(data_test_amazon) :
    data_Lowercase_Amazon_Test.append(data_test_amazon.Sentimen[iterator].lower())
    iterator = iterator + 1

iterator = 0

while iterator < len(data_test_yelp) :
    data_Lowercase_Yelp_Test.append(data_test_yelp.Sentimen[iterator].lower())
    iterator = iterator + 1

In [7]:
data_Preprocessing_Amazon_Train['Lowercase'] = data_Lowercase_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an..."
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...


In [8]:
data_Preprocessing_Amazon_Test['Lowercase'] = data_Lowercase_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...


In [9]:
data_Preprocessing_Yelp_Train['Lowercase'] = data_Lowercase_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei..."


In [10]:
data_Preprocessing_Yelp_Test['Lowercase'] = data_Lowercase_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen..."
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th..."
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan angka

In [11]:
iterator = 0
data_RemoveNumber_Amazon_Train = []

data_RemoveNumber_Yelp_Train = []

while iterator < len(data_Preprocessing_Amazon_Train) :
    data_RemoveNumber_Amazon_Train.append(re.sub(r"\d+", "",data_Preprocessing_Amazon_Train.Lowercase[iterator]))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_RemoveNumber_Yelp_Train.append(re.sub(r"\d+", "",data_Preprocessing_Yelp_Train.Lowercase[iterator]))
    iterator = iterator + 1

In [12]:
iterator = 0
data_RemoveNumber_Amazon_Test = []

data_RemoveNumber_Yelp_Test = []

while iterator < len(data_Preprocessing_Amazon_Test) :
    data_RemoveNumber_Amazon_Test.append(re.sub(r"\d+", "",data_Preprocessing_Amazon_Test.Lowercase[iterator]))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_RemoveNumber_Yelp_Test.append(re.sub(r"\d+", "",data_Preprocessing_Yelp_Test.Lowercase[iterator]))
    iterator = iterator + 1

In [13]:
data_Preprocessing_Amazon_Train['RemoveNumber'] = data_RemoveNumber_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an..."
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [14]:
data_Preprocessing_Amazon_Test['RemoveNumber'] = data_RemoveNumber_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...


In [15]:
data_Preprocessing_Yelp_Train['RemoveNumber'] = data_RemoveNumber_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei..."


In [16]:
data_Preprocessing_Yelp_Test['RemoveNumber'] = data_RemoveNumber_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen..."
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th..."
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan tanda baca

In [17]:
iterator = 0
data_RemovePunctuation_Amazon_Train = []

data_RemovePunctuation_Yelp_Train = []

while iterator < len(data_Preprocessing_Amazon_Train) :
    data_RemovePunctuation_Amazon_Train.append(data_Preprocessing_Amazon_Train.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_RemovePunctuation_Yelp_Train.append(data_Preprocessing_Yelp_Train.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

In [18]:
iterator = 0
data_RemovePunctuation_Amazon_Test = []

data_RemovePunctuation_Yelp_Test = []

while iterator < len(data_Preprocessing_Amazon_Test) :
    data_RemovePunctuation_Amazon_Test.append(data_Preprocessing_Amazon_Test.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_RemovePunctuation_Yelp_Test.append(data_Preprocessing_Yelp_Test.RemoveNumber[iterator].translate(str.maketrans('','', string.punctuation)))
    iterator = iterator + 1

In [19]:
data_Preprocessing_Amazon_Train['RemovePunctuation'] = data_RemovePunctuation_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an...",buyer beware this is a selfpublished book and ...
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...,the worst a complete waste of time typographic...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief i feel i have to write to ...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [20]:
data_Preprocessing_Amazon_Test['RemovePunctuation'] = data_RemovePunctuation_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...,overly complicated being both a us history buf...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...,terrible disappointment parts dont fit my son...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...,didnt hold upvery disappointed i bought this o...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but my sixyearold loves space and to...


In [21]:
data_Preprocessing_Yelp_Train['RemovePunctuation'] = data_RemovePunctuation_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...,i dont know what dr goldberg was like before ...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,im writing this review to give you a heads up ...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...,this place is absolute garbage half of the te...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei...",used to go there for tires brakes etc their p...


In [22]:
data_Preprocessing_Yelp_Test['RemovePunctuation'] = data_RemovePunctuation_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen...",my wife and i used to love arribas til recentl...
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for the food is inexpens...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...",unfortunately yesterdays visit was one of the ...
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday ...
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


# Menghilangkan Non alfabet

In [23]:
iterator = 0

data_Regex_alpabet_only_Amazon_Train = []

data_Regex_alpabet_only_Yelp_Train = []


while iterator < len(data_Preprocessing_Amazon_Train) :
    data_Regex_alpabet_only_Amazon_Train.append(" ".join(re.findall("[a-zA-Z]+", data_Preprocessing_Amazon_Train.RemovePunctuation[iterator])))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Train) :
    data_Regex_alpabet_only_Yelp_Train.append(" ".join(re.findall(r"[a-zA-Z]+", data_Preprocessing_Yelp_Train.RemovePunctuation[iterator])))
    iterator = iterator + 1



In [24]:
iterator = 0

data_Regex_alpabet_only_Amazon_Test = []

data_Regex_alpabet_only_Yelp_Test = []


while iterator < len(data_Preprocessing_Amazon_Test) :
    data_Regex_alpabet_only_Amazon_Test.append(" ".join(re.findall("[a-zA-Z]+", data_Preprocessing_Amazon_Test.RemovePunctuation[iterator])))
    iterator = iterator + 1

iterator = 0

while iterator < len(data_Preprocessing_Yelp_Test) :
    data_Regex_alpabet_only_Yelp_Test.append(" ".join(re.findall(r"[a-zA-Z]+", data_Preprocessing_Yelp_Test.RemovePunctuation[iterator])))
    iterator = iterator + 1

In [25]:
data_Preprocessing_Amazon_Train['Regex'] = data_Regex_alpabet_only_Amazon_Train
data_Preprocessing_Amazon_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,"Buyer beware This is a self-published book, an...","buyer beware this is a self-published book, an...","buyer beware this is a self-published book, an...",buyer beware this is a selfpublished book and ...,buyer beware this is a selfpublished book and ...
1,0,The Worst! A complete waste of time. Typograph...,the worst! a complete waste of time. typograph...,the worst! a complete waste of time. typograph...,the worst a complete waste of time typographic...,the worst a complete waste of time typographic...
2,0,Oh please I guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...,oh please i guess you have to be a romance nov...
3,0,Awful beyond belief! I feel I have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief! i feel i have to write to...,awful beyond belief i feel i have to write to ...,awful beyond belief i feel i have to write to ...
4,0,Another Abysmal Digital Copy Rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...,another abysmal digital copy rather than scrat...


In [26]:
data_Preprocessing_Amazon_Test['Regex'] = data_Regex_alpabet_only_Amazon_Test
data_Preprocessing_Amazon_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,Overly complicated Being both a U.S. history b...,overly complicated being both a u.s. history b...,overly complicated being both a u.s. history b...,overly complicated being both a us history buf...,overly complicated being both a us history buf...
1,0,Terrible Disappointment -- parts don't fit My ...,terrible disappointment -- parts don't fit my ...,terrible disappointment -- parts don't fit my ...,terrible disappointment parts dont fit my son...,terrible disappointment parts dont fit my son ...
2,0,Didn't hold up.....very disappointed I bought ...,didn't hold up.....very disappointed i bought ...,didn't hold up.....very disappointed i bought ...,didnt hold upvery disappointed i bought this o...,didnt hold upvery disappointed i bought this o...
3,0,gene hates jezebel i love JLJ but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...,gene hates jezebel i love jlj but this compila...
4,0,Nice toy but ... My six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but ... my six-year-old loves space a...,nice toy but my sixyearold loves space and to...,nice toy but my sixyearold loves space and to ...


In [27]:
data_Preprocessing_Yelp_Train['Regex'] = data_Regex_alpabet_only_Yelp_Train
data_Preprocessing_Yelp_Train.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,I don't know what Dr. Goldberg was like before...,i don't know what dr. goldberg was like before...,i don't know what dr. goldberg was like before...,i dont know what dr goldberg was like before ...,i dont know what dr goldberg was like before m...
1,0,I'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,i'm writing this review to give you a heads up...,im writing this review to give you a heads up ...,im writing this review to give you a heads up ...
2,0,Owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...,owning a driving range inside the city limits ...
3,0,This place is absolute garbage... Half of the...,this place is absolute garbage... half of the...,this place is absolute garbage... half of the...,this place is absolute garbage half of the te...,this place is absolute garbage half of the tee...
4,0,"Used to go there for tires, brakes, etc. Thei...","used to go there for tires, brakes, etc. thei...","used to go there for tires, brakes, etc. thei...",used to go there for tires brakes etc their p...,used to go there for tires brakes etc their pr...


In [28]:
data_Preprocessing_Yelp_Test['Regex'] = data_Regex_alpabet_only_Yelp_Test
data_Preprocessing_Yelp_Test.head()

,Label,Sentimen,Lowercase,RemoveNumber,RemovePunctuation,Regex
0,0,"My wife and I used to love Arriba's, til recen...","my wife and i used to love arriba's, til recen...","my wife and i used to love arriba's, til recen...",my wife and i used to love arribas til recentl...,my wife and i used to love arribas til recentl...
1,0,You get what you pay for. The food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for. the food is inexpen...,you get what you pay for the food is inexpens...,you get what you pay for the food is inexpensi...
2,0,"Unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...","unfortunately, yesterday's visit was one of th...",unfortunately yesterdays visit was one of the ...,unfortunately yesterdays visit was one of the ...
3,0,I went into the Scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday....,i went into the scottsdale location yesterday ...,i went into the scottsdale location yesterday ...
4,0,It takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...,it takes a lot for me to write a review and bl...


In [29]:
Clean_Amazon_Sentiment_Train = data_Preprocessing_Amazon_Train['Regex']
Clean_Yelp_Sentiment_Train = data_Preprocessing_Yelp_Train['Regex']
Label_Amazon_Train = data_Preprocessing_Amazon_Train['Label']
Label_Yelp_Train = data_Preprocessing_Yelp_Train['Label']

Clean_Amazon_Sentiment_Test = data_Preprocessing_Amazon_Test['Regex']
Clean_Yelp_Sentiment_Test = data_Preprocessing_Yelp_Test['Regex']
Label_Amazon_Test = data_Preprocessing_Amazon_Test['Label']
Label_Yelp_Test = data_Preprocessing_Yelp_Test['Label']

# N-Gram (bigram dan trigram)

## N-gram Amazon

In [30]:
iterator = 0
Vocabulary_Amazon_Train = []
ngram_amazon_train = []

while iterator < len(Clean_Amazon_Sentiment_Train) :
    ngram = 2
    sentence = Clean_Amazon_Sentiment_Train[iterator]
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Train) :
            Vocabulary_Amazon_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_amazon_train.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Train) :
            Vocabulary_Amazon_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1
    vocab = None
    del vocab

    

In [31]:
iterator = 0
Vocabulary_Amazon_Test = []
ngram_amazon_test = []

while iterator < len(Clean_Amazon_Sentiment_Test) :
    ngram = 2
    sentence = Clean_Amazon_Sentiment_Test[iterator]
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Test) :
            Vocabulary_Amazon_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_amazon_test.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Amazon_Test) :
            Vocabulary_Amazon_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1
    vocab = None
    del vocab


## N-gram Yelp

In [32]:
iterator = 0
Vocabulary_Yelp_Train = []
ngram_yelp_train = []

while iterator < len(Clean_Yelp_Sentiment_Train) :
    ngram = 2
    sentence = Clean_Yelp_Sentiment_Train[iterator]
    
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
  
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Train) :
            Vocabulary_Yelp_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_yelp_train.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Train) :
            Vocabulary_Yelp_Train.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1
    

In [33]:
iterator = 0
Vocabulary_Yelp_Test = []
ngram_yelp_test = []

while iterator < len(Clean_Yelp_Sentiment_Test) :
    ngram = 2
    sentence = Clean_Yelp_Sentiment_Test[iterator]
    
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
  
    temp = vocab

    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Test) :
            Vocabulary_Yelp_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
    
    ngram = 3
    vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
    temp = temp + vocab
    ngram_yelp_test.append(temp)
    iterator2 = 0
    while iterator2 < len (vocab):
        if(vocab[iterator2] not in Vocabulary_Yelp_Test) :
            Vocabulary_Yelp_Test.append(vocab[iterator2])
        iterator2 = iterator2 + 1
        
    iterator = iterator + 1

# Me inisialisasi jumlah frekuensi Gram di Vocabulary

In [34]:
WordDictionaryCount_Amazon_Train = []
WordDictionaryCount_Yelp_Train = []

iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Train):
    #Membuat worddictionary gabungan antara vocab amazon test dan train
    WordDictionaryCount_Amazon_Train.append(dict.fromkeys(Vocabulary_Amazon_Train + Vocabulary_Amazon_Test , 0))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Train):
    WordDictionaryCount_Yelp_Train.append(dict.fromkeys(Vocabulary_Yelp_Train + Vocabulary_Yelp_Test, 0))
    iterator = iterator + 1

In [35]:
WordDictionaryCount_Amazon_Test = []
WordDictionaryCount_Yelp_Test = []

iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Test):
    WordDictionaryCount_Amazon_Test.append(dict.fromkeys(Vocabulary_Amazon_Test + Vocabulary_Amazon_Train, 0))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Test):
    WordDictionaryCount_Yelp_Test.append(dict.fromkeys(Vocabulary_Yelp_Test + Vocabulary_Yelp_Train, 0))
    iterator = iterator + 1

In [36]:
iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Train) :
    for gram in ngram_amazon_train[iterator] :
        WordDictionaryCount_Amazon_Train[iterator][gram] += 1
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Train) :
    for gram in ngram_yelp_train[iterator] :
        WordDictionaryCount_Yelp_Train[iterator][gram] += 1
    iterator = iterator + 1

In [37]:
iterator = 0
while iterator < len(Clean_Amazon_Sentiment_Test) :
    for gram in ngram_amazon_test[iterator] :
        WordDictionaryCount_Amazon_Test[iterator][gram] += 1
    iterator = iterator + 1
    
iterator = 0
while iterator < len(Clean_Yelp_Sentiment_Test) :
    for gram in ngram_yelp_test[iterator] :
        WordDictionaryCount_Yelp_Test[iterator][gram] += 1
    iterator = iterator + 1

In [38]:
pd.DataFrame([WordDictionaryCount_Amazon_Train[0],WordDictionaryCount_Amazon_Train[1]])

,bu,uy,ye,er,r,b,be,ew,wa,ar,...,hue,wrh,igf,tnb,sgn,gnr,nr,ydl,yov,jae
0,1,1,1,11,9,13,5,3,4,7,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2,5,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [39]:
pd.DataFrame([WordDictionaryCount_Amazon_Test[0],WordDictionaryCount_Amazon_Test[1]])

,ov,ve,er,rl,ly,y,c,co,om,mp,...,xd,omw,pyb,cyo,bsb,myu,sez,fja,kav,ukw
0,3,7,14,1,2,13,7,2,2,2,...,0,0,0,0,0,0,0,0,0,0
1,0,1,8,0,1,7,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
pd.DataFrame([WordDictionaryCount_Yelp_Train[0],WordDictionaryCount_Yelp_Train[1]])

,i,d,do,on,nt,t,k,kn,no,ow,...,qt,qt,lny,fep,cpn,mds,wfs,itv,agb,ej
0,4,11,6,13,7,19,1,1,5,1,...,0,0,0,0,0,0,0,0,0,0
1,10,7,6,10,13,21,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [41]:
pd.DataFrame([WordDictionaryCount_Yelp_Test[0],WordDictionaryCount_Yelp_Test[1]])

,my,y,w,wi,if,fe,e,a,an,nd,...,wyu,nhg,hgn,nwc,sez,fbe,yec,nww,zeg,ypu
0,3,14,27,5,3,4,44,30,14,10,...,0,0,0,0,0,0,0,0,0,0
1,0,3,3,1,0,0,9,4,4,3,...,0,0,0,0,0,0,0,0,0,0


## Menghitung nilai TF pada setiap data Amazon dan Yelp

In [42]:
def computeTF(wordDict, ngram):
    tfDict = {}
    ngramLength = len(ngram)
    for word, count in wordDict.items():
        tfDict[word] = count/float(ngramLength)
    return tfDict

In [43]:
print(len(WordDictionaryCount_Amazon_Train))
print(len(WordDictionaryCount_Yelp_Train))

7500
7500


In [44]:
Tf_Sentimen_Amazon_Train = []
Tf_Sentimen_Yelp_Train = []
Tf_Sentimen_Amazon_Test = []
Tf_Sentimen_Yelp_Test = []

iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Train):
    Tf_Sentimen_Amazon_Train.append(computeTF(WordDictionaryCount_Amazon_Train[iterator], ngram_amazon_train[iterator]))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Test):
    Tf_Sentimen_Amazon_Test.append(computeTF(WordDictionaryCount_Amazon_Test[iterator], ngram_amazon_test[iterator]))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    Tf_Sentimen_Yelp_Train.append(computeTF(WordDictionaryCount_Yelp_Train[iterator], ngram_yelp_train[iterator]))
    iterator = iterator + 1
    
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Test):
    Tf_Sentimen_Yelp_Test.append(computeTF(WordDictionaryCount_Yelp_Test[iterator], ngram_yelp_test[iterator]))
    iterator = iterator + 1
    

## Menghitung nilai IDF pada setiap data Amazon dan Yelp

In [45]:
import math
def computeIDF(WordDict):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDict)
    
    idfDict = dict.fromkeys(WordDict[0].keys(), 0)
    for doc in WordDict:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        number_of_document_with_term_t_in_it[word] = float(val)
        idfDict[word] = math.log(N / float(val))
        
    return idfDict, number_of_document_with_term_t_in_it

IDF_Amazon_Train, number_of_document_with_term_t_Amazon_Train = computeIDF(WordDictionaryCount_Amazon_Train)
IDF_Yelp_Train, number_of_document_with_term_t_in_Yelp_Train = computeIDF(WordDictionaryCount_Yelp_Train)

IDF_Amazon_Test, number_of_document_with_term_t_Amazon_Test = computeIDF(WordDictionaryCount_Amazon_Test)
IDF_Yelp_Test, number_of_document_with_term_t_in_Yelp_Test = computeIDF(WordDictionaryCount_Yelp_Test)

ZeroDivisionError: float division by zero

## Menghitung nilai TF-IDF pada setiap data Amazon dan Yelp

In [ ]:
def computeTFIDF(Tf_Sentimen_Amazon, idfs):
    tfidf = {}
    for word, val in Tf_Sentimen_Amazon.items():
        tfidf[word] = val*idfs[word]
    return tfidf


TF_IDF_Amazon_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Train):
    TF_IDF_Amazon_Train.append(computeTFIDF(Tf_Sentimen_Amazon_Train[iterator], IDF_Amazon_Train))
    iterator = iterator + 1
    
TF_IDF_Amazon_Test = []
iterator = 0
while iterator < len(WordDictionaryCount_Amazon_Test):
    TF_IDF_Amazon_Test.append(computeTFIDF(Tf_Sentimen_Amazon_Test[iterator], IDF_Amazon_Test))
    iterator = iterator + 1
    
TF_IDF_Yelp_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    TF_IDF_Yelp_Train.append(computeTFIDF(Tf_Sentimen_Yelp_Train[iterator], IDF_Yelp_Train))
    iterator = iterator + 1
    
TF_IDF_Yelp_Test = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Test):
    TF_IDF_Yelp_Test.append(computeTFIDF(Tf_Sentimen_Yelp_Test[iterator], IDF_Yelp_Test))
    iterator = iterator + 1

## Menampung nilai TF-IDF pada variabel Vectorizer

In [ ]:
Vectorizer_Amazon_Train = []
iterator  = 0
while iterator < len(TF_IDF_Amazon_Train):
    Vectorizer_Amazon_Train.append(list(TF_IDF_Amazon_Train[iterator].values()))
    iterator = iterator + 1
    
Vectorizer_Yelp_Train = []
iterator  = 0
while iterator < len(TF_IDF_Yelp_Train):
    Vectorizer_Yelp_Train.append(list(TF_IDF_Yelp_Train[iterator].values()))
    iterator = iterator + 1
    
Vectorizer_Amazon_Test = []
iterator  = 0
while iterator < len(TF_IDF_Amazon_Test):
    Vectorizer_Amazon_Test.append(list(TF_IDF_Amazon_Test[iterator].values()))
    iterator = iterator + 1
    
Vectorizer_Yelp_Test = []
iterator  = 0
while iterator < len(TF_IDF_Yelp_Test):
    Vectorizer_Yelp_Test.append(list(TF_IDF_Yelp_Test[iterator].values()))
    iterator = iterator + 1
    

## Data Frame Vectorizer Amazon

In [ ]:
DataFrame_Amazon_Train = pd.DataFrame(Vectorizer_Amazon_Train, columns = Vocabulary_Amazon_Train)
print (DataFrame_Amazon_Train)

## Dataframe Vectorizer Yelp

In [ ]:
DataFrame_Yelp_Train = pd.DataFrame(Vectorizer_Yelp_Train, columns = Vocabulary_Yelp_Train)
print (DataFrame_Yelp_Train)

# Jumlah data label positif dan negatif

In [ ]:
print("Data positif negatif amazon")
DataFrame_Label = pd.DataFrame(Label_Amazon_Train)
print(DataFrame_Label['Label'].value_counts())

print("\n")

print("Data positif negatif yelp")
DataFrame_Label = pd.DataFrame(Label_Yelp_Train)
print(DataFrame_Label['Label'].value_counts())

In [ ]:
print("Amazon data")
Vectorizer_Amazon_Train = np.array(Vectorizer_Amazon_Train)
Vectorizer_Yelp_Train = np.array(Vectorizer_Yelp_Train)
Vectorizer_Amazon_Test = np.array(Vectorizer_Amazon_Test)
Vectorizer_Yelp_Test = np.array(Vectorizer_Yelp_Test)

Label_Amazon_Train = np.array(Label_Amazon_Train)
Label_Yelp_Train = np.array(Label_Yelp_Train)
Label_Amazon_Test = np.array(Label_Amazon_Test)
Label_Yelp_Test = np.array(Label_Yelp_Test)

print("data train shape : {}".format(Vectorizer_Amazon_Train.shape))
print("data test shape : {}".format(Vectorizer_Amazon_Test.shape))
print("label train shape : {}".format(Label_Amazon_Train.shape))
print("label test shape : {}".format(Label_Amazon_Test.shape))

print("\nYelp data")
print("data train shape : {}".format(Vectorizer_Yelp_Train.shape))
print("data test shape : {}".format(Vectorizer_Yelp_Test.shape))
print("label train shape : {}".format(Label_Yelp_Train.shape))
print("label test shape : {}".format(Label_Yelp_Test.shape))

In [ ]:
# from sklearn.decomposition import PCA

# dr = PCA(n_components = 300)
# dr.fit(data_train_amazon)
# data_train_amazon = dr.transform(data_train_amazon)
# data_test_amazon = dr.transform(data_test_amazon)

# Skor Akurasi Amazon dan Yelp

In [ ]:
RF_Classifier_Amazon = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Amazon.fit(Vectorizer_Amazon_Train, Label_Amazon_Train)
print("skor RF_Amazon: {}".format(RF_Classifier_Amazon.score(Vectorizer_Amazon_Test, Label_Amazon_Test)))

# RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
#                                        bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
# RF_Classifier_Yelp.fit(data_train_yelp, label_train_yelp)
# print("skor RF_Yelp: {}".format(RF_Classifier_Yelp.score(data_test_yelp, label_test_yelp)))


In [ ]:
number_of_document_with_term_t_Amazon

In [ ]:
Vocabulary_Intersection = []
iterator = 0
while iterator < len(Vocabulary_Yelp) :
    if Vocabulary_Yelp[iterator] in Vocabulary_Amazon :
        Vocabulary_Intersection.append(Vocabulary_Yelp[iterator])
    iterator = iterator + 1
    


In [ ]:
def RecomputeIDF(WordDictAmazon, WordDict, additional_term_value, Vocabulary_Intersection):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDict) + len(WordDictAmazon)
    print(len(WordDict))
    print(len(WordDictAmazon))
    idfDict = dict.fromkeys(WordDict[0].keys(), 0)
    for doc in WordDict:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if word in Vocabulary_Intersection :
            print(additional_term_value[word])
            val = val + additional_term_value[word]
        idfDict[word] = math.log(N / float(val))
        
    return idfDict


New_IDF_Yelp = RecomputeIDF(WordDictionaryCount_Amazon,WordDictionaryCount_Yelp, number_of_document_with_term_t_Amazon, Vocabulary_Intersection)

In [ ]:
New_TF_IDF_Yelp = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp):
    New_TF_IDF_Yelp.append(computeTFIDF(Tf_Sentimen_Yelp[iterator], New_IDF_Yelp))
    iterator = iterator + 1

In [ ]:
New_Vectorizer_Yelp = []
iterator  = 0
while iterator < len(New_TF_IDF_Yelp):
    New_Vectorizer_Yelp.append(list(New_TF_IDF_Yelp[iterator].values()))
    iterator = iterator + 1

In [ ]:
new_data_train_yelp, new_data_test_yelp, new_label_train_yelp, new_label_test_yelp = train_test_split((New_Vectorizer_Yelp), Label_Yelp, random_state = 0, test_size = 0.99)

In [ ]:
New_RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
New_RF_Classifier_Yelp.fit(new_data_train_yelp, new_label_train_yelp)
print("skor RF_Yelp: {}".format(New_RF_Classifier_Yelp.score(new_data_test_yelp, new_label_test_yelp)))